In [ ]:
from __future__ import print_function
import torch.nn as nn
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import torch.nn.functional as F

torch.manual_seed(1992)

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> 1. GAN Introduction </h1>

![gan](https://learn-neural-networks.com/wp-content/uploads/2020/04/gan.jpg)

The core idea of a GAN is based on the "indirect" training through the discriminator, another neural network that is able to tell how much an input is "realistic", which itself is also being updated dynamically This basically means that the generator is not trained to minimize the distance to a specific image, but rather to fool the discriminator. This enables the model to learn in an unsupervised manner.(Wiki)<br>
<br>
In this notebook I will share ready to use GAN Generator
1. Intialize Generator instance
2. Create the random noise
3. Input random noise to generator
4. Yield the generated images
5. Create unlimited COTS

**When you will be interested in the training routine I will consider to create another notebook where whole training process will be explained**<br>
Gan is the interesting approach in data augumentation field.<br>
I have also presented the Neural Style Transfer [Notebook Here](https://www.kaggle.com/marcinstasko/cots-neuralstyle-transfer-pytorch-augumentation)


<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> 2. Create Generator Instance </h1>


In [ ]:
# Define the Generator Network
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input is the latent vector Z.
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params['ngf']*8,
            kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(params['ngf']*8)

        # Input Dimension: (ngf*8) x 4 x 4
        self.tconv2 = nn.ConvTranspose2d(params['ngf']*8, params['ngf']*4,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ngf']*4)

        # Input Dimension: (ngf*4) x 8 x 8
        self.tconv3 = nn.ConvTranspose2d(params['ngf']*4, params['ngf']*2,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ngf']*2)

        # Input Dimension: (ngf*2) x 16 x 16
        self.tconv4 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'],
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ngf'])

        # Input Dimension: (ngf) * 32 * 32
        self.tconv5 = nn.ConvTranspose2d(params['ngf'], params['nc'],
            4, 2, 1, bias=False)
        #Output Dimension: (nc) x 64 x 64

    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))

        x = F.tanh(self.tconv5(x))

        return x

In [ ]:
# Input necessary parameters (established during training)
params = {
    'nc' : 3,    # Number of channles (3-colour)
    'nz' : 100,  # The input size to the generator (latent vector).
    'ngf' : 64,  # Feature maps (For training)
}


In [ ]:
MODEL_PATH = '../input/cots-ganv1/COTS_GAN_v1.pth'
netG = Generator(params).cuda()
model = torch.load(MODEL_PATH)
netG.load_state_dict(model['generator'])
netG.eval()

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> 3. Generate Images </h1>


In [ ]:
from tqdm import tqdm_notebook as tqdm
# How many images in batch
im_batch_size = 40

# How many images in total
n_images=40

plt.rcParams['figure.figsize'] = (4*n_images, n_images)

device = torch.device('cuda')
plt.ion() # plt interactive mode 

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    # Generate Random Noise for generator
    gen_z = torch.randn(im_batch_size, params['nz'], 1, 1, device=device)
    
    # Input noise to the generator instance
    gen_images = netG(gen_z)
    
    # Create grind and plot the results
    grid = vutils.make_grid(gen_images, nrow=4, padding=10,normalize=True)
    cpu_grid = grid.to("cpu")#.clone().detach()
    cpu_permuted = np.transpose(cpu_grid,(1,2,0))
    plt.imshow(cpu_permuted)
    
    # If you want to save the image, 
    # Change "./" to your location if other than working directory
    
    vutils.save_image(gen_images.data, '{}/COTS_GAN1{}.jpg'.format("./", i_batch), normalize=True)

In [ ]:
!ls ./

**If you like the code please upvote and comment :)**